# __CNN__

We decide to use CNN(convolutional neural networks) for the task of guitar tablature estimation. The previous work of Andrew Wiggins and Youngmoo Kim showed that CNNs have shown promise for translating guitar audios to tabs, and the use of CNNs has also been explored for various other tasks within music information retrieval such as musical tempo estimation, key classification, singing voice detection, and instrument classification. It is proven that CNN is a powerful tool for the purpose of our study.

## __Import libraries__

In [1]:
# Import required packages 
#tensorflow
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import layers

#various
import datetime
import pathlib
import IPython.display as display
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from PIL import Image

#keras
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import ReduceLROnPlateau
from keras import backend as K


#sklearn
from sklearn.model_selection import train_test_split


# Load the TensorBoard notebook extension
%load_ext tensorboard

RSEED = 42

In [2]:
#Clear any logs from previous runs
!rm -rf ./logs/

In [3]:
# Check for Tensorflow version
print(tf.__version__)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

2.9.0


## __"Write Python Script" function__

`%%write_and_run image_modeling.py` is the call of the register cell magic from below in 'w' mode (default). It writes the imports at the beginning of the `image_modeling.py`.

In [4]:
'''# Let's make some dark cell magic. Why not!

from IPython.core.magic import register_cell_magic

@register_cell_magic
def write_and_run(line, cell):
    argz = line.split()
    file = argz[-1]
    mode = 'w'
    if len(argz) == 2 and argz[0] == '-a':
        mode = 'a'
        print("Appended to file ", file)
    else:
        print('Written to file:', file)
    with open(file, mode) as f:
        f.write(cell.format(**globals()))        
    get_ipython().run_cell(cell)'''

'# Let\'s make some dark cell magic. Why not!\n\nfrom IPython.core.magic import register_cell_magic\n\n@register_cell_magic\ndef write_and_run(line, cell):\n    argz = line.split()\n    file = argz[-1]\n    mode = \'w\'\n    if len(argz) == 2 and argz[0] == \'-a\':\n        mode = \'a\'\n        print("Appended to file ", file)\n    else:\n        print(\'Written to file:\', file)\n    with open(file, mode) as f:\n        f.write(cell.format(**globals()))        \n    get_ipython().run_cell(cell)'

## __Define Input Shapes__

In [5]:
#Import variables from our file
FRAME_HEIGHT = 192
FRAME_WIDTH = 9
N_CLASSES = 21
N_STRINGS = 6
BATCH_SIZE = 128
EPOCHS = 8

TRAIN_PATH = 'our trainset path.csv'
EVAL_PATH = 'our evalset path.csv'
TEST_PATH = 'our testset path.csv'

#TRAINING_SIZE = !wc -l < flowers_train.csv
#TRAINING_STEPS = int(TRAINING_SIZE[0]) // BATCH_SIZE




Using Tensorboard to monitor our results:

In [6]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6008 (pid 98308), started 1 day, 1:29:15 ago. (Use '!kill 98308' to kill it.)

## __Load Data__

In [7]:
X = np.load('/Users/ivy/neuefische/swizzle/data/output/02_BN1-129-Eb_solo_hex_cln_data.npz')
y = np.load('/Users/ivy/neuefische/swizzle/data/output/02_BN1-129-Eb_solo_hex_cln_labels.npz')

In [8]:
X['arr_0']

array([[[  0.        ,   0.        ,   0.        , ..., -44.34732056,
         -44.10077667, -43.91782379],
        [  0.        ,   0.        ,   0.        , ..., -48.74607849,
         -49.20463562, -49.76281738],
        [  0.        ,   0.        ,   0.        , ..., -48.10150909,
         -48.1938858 , -48.34008789],
        ...,
        [  0.        ,   0.        ,   0.        , ..., -80.        ,
         -80.        , -80.        ],
        [  0.        ,   0.        ,   0.        , ..., -80.        ,
         -80.        , -80.        ],
        [  0.        ,   0.        ,   0.        , ..., -80.        ,
         -80.        , -80.        ]],

       [[  0.        ,   0.        ,   0.        , ..., -44.10077667,
         -43.91782379, -43.80202484],
        [  0.        ,   0.        ,   0.        , ..., -49.20463562,
         -49.76281738, -50.40135193],
        [  0.        ,   0.        ,   0.        , ..., -48.1938858 ,
         -48.34008789, -48.52977753],
        ...,


## __Do train & test split__

In [9]:
x_train, x_test, y_train, y_test = train_test_split(X['arr_0'], y['arr_0'], test_size= 0.3, random_state= RSEED )

Do this again and split the train set, so we can produce a validation set

In [10]:

x_train,x_validate,y_train,y_validate = train_test_split(x_train,y_train, test_size = 0.2,random_state = RSEED)

## __Reshape__

Now we shall build the CNN!

Actually, no.

We'll have to do one final step of reshaping ALL the data before we go forward to build it.

Why do we need to reshape the data? Because each data point is in the form of a single row. In this CNN example, we're going to work with IMAGES.

The CNN needs images to extract coarse and fine features so that it can train and classify them.

In [26]:
im_shape = (FRAME_HEIGHT, FRAME_WIDTH, 1)
x_train = x_train.reshape(x_train.shape[0], *im_shape) # Python TIP :the * operator unpacks the tuple
x_test = x_test.reshape(x_test.shape[0], *im_shape)

## __Define functions__

In [12]:
#defining softmax function

def softmax_by_string(t):
        sh = K.shape(t)
        string_sm = []
        for i in range(N_STRINGS):
            string_sm.append(K.expand_dims(K.softmax(t[:,i,:]), axis=1))
        return K.concatenate(string_sm, axis=1)


## Building our CNN Model

In [27]:
def cnn_swizzle_model():       
        swizzle_model = tf.keras.Sequential()
        swizzle_model.add(tf.keras.layers.InputLayer(input_shape=[FRAME_HEIGHT, FRAME_WIDTH, 1], name='image'))
        swizzle_model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3),activation='relu'))
        swizzle_model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
        swizzle_model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
        swizzle_model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
        swizzle_model.add(tf.keras.layers.Dropout(0.25))   
        swizzle_model.add(tf.keras.layers.Flatten())
        swizzle_model.add(tf.keras.layers.Dense(128, activation='relu'))
        swizzle_model.add(tf.keras.layers.Dropout(0.5))
        swizzle_model.add(tf.keras.layers.Dense(126, activation='relu'))
        swizzle_model.add(tf.keras.layers.Dense(N_CLASSES * N_STRINGS)) # no activation
        swizzle_model.add(tf.keras.layers.Reshape((N_CLASSES, N_STRINGS)))
        swizzle_model.add(tf.keras.layers.Activation(softmax_by_string))
        return swizzle_model

## Define model metrics for the cnn

In [14]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

#RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0) 
#adadelta

In [15]:
swizzle_model = cnn_swizzle_model()

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2023-01-03 11:42:15.626388: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-01-03 11:42:15.626665: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [23]:
swizzle_model = cnn_swizzle_model()
swizzle_model.compile(loss =tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer= optimizer, metrics =['accuracy'])
#we use sparse_categorical_crossentropy because we have a one-hot-encoding for every note. so we have only 0 and 1. If we have more
#numbers like this, we have to use categorical_crossentropy.
#we use accuracy because we want to have a good overall performance of the model. Besides that for us, every tone 
#has the same importance so all classes have same importance.

## Use annealer to decrease learning rate in each epoch

With the ReduceLROnPlateau function from Keras.callbacks, we choose to reduce the LR by half if the accuracy is not improved after 3 epochs.

In [17]:
# Set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=0.5, min_lr=0.0001)

## __Train CNN__

In [28]:
history = swizzle_model.fit(
    x_train,
    y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    verbose=1,
    validation_data=(x_validate,y_validate),
    callbacks=[learning_rate_reduction],
)

Epoch 1/8


InvalidArgumentError: Graph execution error:

Detected at node 'sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits' defined at (most recent call last):
    File "/Users/ivy/.pyenv/versions/3.9.8/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Users/ivy/.pyenv/versions/3.9.8/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/Users/ivy/neuefische/swizzle/.venv/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/Users/ivy/neuefische/swizzle/.venv/lib/python3.9/site-packages/traitlets/config/application.py", line 992, in launch_instance
      app.start()
    File "/Users/ivy/neuefische/swizzle/.venv/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 711, in start
      self.io_loop.start()
    File "/Users/ivy/neuefische/swizzle/.venv/lib/python3.9/site-packages/

In [ ]:
#show plots for our loss function and the accurancy
plt.figure(figsize=(10, 10))

plt.subplot(2, 2, 1)
plt.plot(history.history['loss'], label='Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Training - Loss Function')

plt.subplot(2, 2, 2)
plt.plot(history.history['acc'], label='Accuracy')
plt.plot(history.history['val_acc'], label='Validation Accuracy')
plt.legend()
plt.title('Train - Accuracy')

In [ ]:
#print results of our swizzle model metrics
score = swizzle_model.evaluate(x_test,y_test,verbose=0)
print('Test Loss : {:.4f}'.format(score[0]))
print('Test Accuracy : {:.4f}'.format(score[1]))

In [ ]:
swizzle_model.save('cnn_Tabs_of_songs_with_20_epochs.h5')